In [59]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)

In [85]:
lob = pd.read_csv('./data/neural_network_predictions/predictions_06_18.csv')

In [86]:
td = pd.to_timedelta(lob['timestamps'],'s')
lob = lob.set_index(td)

In [87]:
lob['s'] = lob['a'] - lob['b']

In [88]:
dfs = []
dfs.append(lob)

In [89]:
def trading_algorithm(x, df):
            if x[-1] == 2:
                if df.s[x.index[-1]] <= 3: # x = 3
                    if df.b[x.index[0]] - df.a[x.index[-1]] > 0:
                        return df.b[x.index[0]] - df.a[x.index[-1]]
                    else:
                        return np.NaN
                else:
                    return 0
            if x[-1] == 1:
                return 0
            if x[-1] == 0:
                if df.s[x.index[-1]] <= 3: # x = 3
                    if df.b[x.index[-1]] - df.a[x.index[0]] > 0:
                        return df.b[x.index[-1]] - df.a[x.index[0]]
                    else:
                        return np.NaN
                else:
                    return 0

In [90]:
PROFITS_FOR_EACH_DAY_AS_TEST_SET = []

for i in range(len(dfs)):

    lob_profit = dfs[i]

    lob_profit_r = lob_profit[::-1]
    lob_profit_r['Profit'] = lob_profit_r['Model Prediction'].rolling('10s').apply(lambda x: trading_algorithm(x, lob_profit_r))
    lob_profit = lob_profit.join(lob_profit_r['Profit'])
    
    display(lob_profit)

    profit = lob_profit['Profit'].sum()
    sell_inv_mean = lob_profit.b[np.logical_and(lob_profit['Profit'].isnull(), lob_profit['Model Prediction'] == 0)].mean()
    sell_inv_count = lob_profit.b[np.logical_and(lob_profit['Profit'].isnull(), lob_profit['Model Prediction'] == 0)].count()
    buy_inv_mean = lob_profit.a[np.logical_and(lob_profit['Profit'].isnull(), lob_profit['Model Prediction'] == 2)].mean()
    buy_inv_count = lob_profit.a[np.logical_and(lob_profit['Profit'].isnull(), lob_profit['Model Prediction'] == 2)].count()
    net_inv_loss = (buy_inv_mean - sell_inv_mean) * min(buy_inv_count, sell_inv_count)
    day_avg_bid_price = lob_profit.b.mean()
    day_avg_ask_price = lob_profit.a.mean()
    if buy_inv_count > sell_inv_count:
        inv_loss_rem_at_day_avg = (buy_inv_mean-day_avg_bid_price)*(buy_inv_count-sell_inv_count)
    else:
        inv_loss_rem_at_day_avg = (day_avg_ask_price-sell_inv_mean)*(sell_inv_count-buy_inv_count)
    TOTAL_PROFIT = profit - net_inv_loss - inv_loss_rem_at_day_avg

    PROFITS_FOR_EACH_DAY_AS_TEST_SET.append(TOTAL_PROFIT)

    print(f'Day: s <= 3\nDirect Trading Profit from correct predictions:',profit,"\nSell Inventory:",sell_inv_count,"at Avg Price",sell_inv_mean,"\nBuy Inventory:",buy_inv_count,"at Avg Price",buy_inv_mean,"\nNet Inventory Loss:",net_inv_loss,"\nDay Average Bid Price:",day_avg_bid_price,"\nLoss on remaining inventory if sold at day average:",inv_loss_rem_at_day_avg,"\nOVERALL DAY PROFIT:",TOTAL_PROFIT)

C:\Users\Morg\AppData\Local\Temp\ipykernel_23612\657546327.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lob_profit_r['Profit'] = lob_profit_r['Model Prediction'].rolling('10s').apply(lambda x: trading_algorithm(x, lob_profit_r))


,Unnamed: 0,timestamps,b,a,Model Prediction,s,Profit
timestamps,,,,,,,
0 days 00:00:16.492000,0,16.492,97.0,100.0,2,3.0,NaN
0 days 00:00:16.523000,1,16.523,97.0,100.0,2,3.0,NaN
0 days 00:00:16.678000,2,16.678,97.0,100.0,2,3.0,NaN
0 days 00:00:16.895000,3,16.895,97.0,100.0,2,3.0,NaN
0 days 00:00:17.019000,4,17.019,97.0,103.0,2,6.0,0.0
...,...,...,...,...,...,...,...
0 days 08:29:59.232000,325575,30599.232,104.0,178.0,1,74.0,0.0
0 days 08:29:59.263000,325576,30599.263,105.0,178.0,1,73.0,0.0
0 days 08:29:59.387000,325577,30599.387,105.0,175.0,1,70.0,0.0


Day: s <= 3
Direct Trading Profit from correct predictions: 57782.0 
Sell Inventory: 1 at Avg Price 105.0 
Buy Inventory: 4187 at Avg Price 102.77477907809887 
Net Inventory Loss: -2.2252209219011263 
Day Average Bid Price: 104.58995963704054 
Loss on remaining inventory if sold at day average: -7598.3458197298 
OVERALL DAY PROFIT: 65382.5710406517


In [52]:
lob_profit[lob_profit['Profit'] != 0].head(300)

,timestamps,b,a,Model Prediction,s,Profit
timestamps,,,,,,
0 days 00:01:04.852000,64.852,107.0,108.0,2,1.0,NaN
0 days 00:01:04.914000,64.914,107.0,108.0,2,1.0,NaN
0 days 00:01:05.038000,65.038,107.0,108.0,2,1.0,NaN
0 days 00:01:18.058000,78.058,101.0,102.0,2,1.0,NaN
0 days 00:01:18.182000,78.182,101.0,102.0,2,1.0,NaN
0 days 00:01:18.213000,78.213,101.0,102.0,2,1.0,NaN
0 days 00:01:18.275000,78.275,101.0,102.0,2,1.0,NaN
0 days 00:01:18.399000,78.399,101.0,102.0,2,1.0,1.0
0 days 00:01:18.585000,78.585,101.0,102.0,2,1.0,1.0
